## Data for Figure 9

#### Based on Elise Olson's https://github.com/SalishSeaCast/analysis-elise-2/blob/master/notebooks/modelEqs/NewRateComparison.ipynb notebook

In [1]:
import numpy as np
import netCDF4 as nc
import matplotlib.pyplot as plt
from salishsea_tools import bio_tools as bt, places
import xarray as xr
import os
import glob
%matplotlib inline

In [2]:
nml=bt.load_nml_bio(resDir='/ocean/eolson/MEOPAR/NEMO-3.6-code/NEMOGCM/CONFIG/SalishSeaCast/EXP00/',
                 nmlname='nampisprod',bioRefName='namelist_smelt_cfg_HC201905equiv',bioCfgName='namelist_smelt_cfg_HC201905equiv')

In [3]:
nml

Namelist([('zz_rate_r_diat', 6.0495e-05),
          ('zz_rate_r_myri', 2.22e-05),
          ('zz_rate_r_nano', 2.109e-05),
          ('zz_rate_maxtemp_diat', 26.0),
          ('zz_rate_maxtemp_myri', 31.0),
          ('zz_rate_maxtemp_nano', 31.0),
          ('zz_rate_temprange_diat', 14.0),
          ('zz_rate_temprange_myri', 13.0),
          ('zz_rate_temprange_nano', 13.0),
          ('zz_rate_iopt_diat', 45.0),
          ('zz_rate_iopt_myri', 37.0),
          ('zz_rate_iopt_nano', 10.0),
          ('zz_rate_gamma_diat', 0.0),
          ('zz_rate_gamma_myri', 0.0),
          ('zz_rate_gamma_nano', 0.0),
          ('zz_rate_k_si_diat', 2.2),
          ('zz_rate_k_si_myri', 0.0),
          ('zz_rate_k_si_nano', 0.0),
          ('zz_rate_kapa_diat', 1.0),
          ('zz_rate_kapa_myri', 0.5),
          ('zz_rate_kapa_nano', 0.3),
          ('zz_rate_k_diat', 2.0),
          ('zz_rate_k_myri', 0.5),
          ('zz_rate_k_nano', 0.2),
          ('zz_rate_si_ratio_diat', 1.8),
          

In [4]:
def phyto_Tdep_Factor(TT, zz_rate_maxtemp, zz_rate_temprange):
    if hasattr(TT,'__len__'): # assume 1-d array or similar and return array
        return np.array([phyto_Tdep_Factor(el,zz_rate_maxtemp, zz_rate_temprange) for el in TT])
    else:
        return np.exp(0.07 * (TT - 20)) * min(max((zz_rate_maxtemp - TT), 0.0),zz_rate_temprange) / (zz_rate_temprange + 1e-10)

In [5]:
def calc_T_Factors(TT,nampisprod):
    Tdep_Diat=phyto_Tdep_Factor(TT,nampisprod['zz_rate_maxtemp_diat'],nampisprod['zz_rate_temprange_diat'])
    Tdep_Myri=phyto_Tdep_Factor(TT,nampisprod['zz_rate_maxtemp_myri'],nampisprod['zz_rate_temprange_myri'])
    Tdep_Nano=phyto_Tdep_Factor(TT,nampisprod['zz_rate_maxtemp_nano'],nampisprod['zz_rate_temprange_nano'])
    return Tdep_Diat, Tdep_Myri, Tdep_Nano

### Import Temperature Files

In [6]:
# Original CY 

monthly_array_temp_orig_slice = np.zeros([14,12,50,50])
# Load monthly averages
mask = xr.open_dataset('/data/eolson/results/MEOPAR/NEMO-forcing-new/grid/mesh_mask201702.nc')
slc = {'y': slice(450,500), 'x': slice(250,300)}
e3t, tmask = [mask[var].isel(z=slice(None, 27),**slc).values for var in ('e3t_0', 'tmask')]
years, variables = range(2007, 2021), ['votemper']
# Temporary list dict
data = {}
# Permanent aggregate dict
aggregates = {var: {} for var in variables}
monthlydat = {var: {} for var in variables}

### 2008 using higher temperature threshold       
# Add experiment year
for year in [2008]:
    # Initialize lists
    for var in variables: data[var] = []
    # Load monthly averages
    for month in range(1, 13):
        datestr = f'{year}{month:02d}'
        prefix = f'/data/sallen/results/MEOPAR/v201905r/SalishSea_1m_{datestr}_{datestr}'
        
        with xr.open_dataset(prefix + '_grid_T.nc') as ds:
            q = ds.votemper.isel(deptht=0, **slc).values
            q2 = q[0,:,:]
            monthly_array_temp_orig_slice[year-2007,month-1,:,:] = q2 #year2007 is index 0 along 1st dimension
            for var in ['votemper']:
                data[var].append(ds.votemper.isel(deptht=0, **slc).values)
    # Concatenate months
    for var in variables: aggregates[var][year] = np.concatenate(data[var]).mean(axis=0)


In [7]:
monthly_array_temp_orig_slice[monthly_array_temp_orig_slice == 0 ] = np.nan
monthly_array_temp_orig_slicemean = \
np.nanmean(np.nanmean(monthly_array_temp_orig_slice, axis = 2),axis = 2)
print(np.shape(monthly_array_temp_orig_slicemean))

(14, 12)


/tmp/ipykernel_3793737/1734163470.py:3: RuntimeWarning: Mean of empty slice
  np.nanmean(np.nanmean(monthly_array_temp_orig_slice, axis = 2),axis = 2)


In [8]:
## Experiment 8

monthly_array_temp_exp_slice = np.zeros([14,12,50,50])
# Load monthly averages
mask = xr.open_dataset('/data/eolson/results/MEOPAR/NEMO-forcing-new/grid/mesh_mask201702.nc')
slc = {'y': slice(450,500), 'x': slice(250,300)}
e3t, tmask = [mask[var].isel(z=slice(None, 27),**slc).values for var in ('e3t_0', 'tmask')]
years, variables = range(2007, 2021), ['votemper']
# Temporary list dict
data = {}
# Permanent aggregate dict
aggregates = {var: {} for var in variables}
monthlydat = {var: {} for var in variables}

### 2008 using higher temperature threshold       
# Add experiment year
for year in [2008]:
    # Initialize lists
    for var in variables: data[var] = []
    # Load monthly averages
    for month in range(1, 7):
        datestr = f'{year}{month:02d}'
        prefix = f'/data/sallen/results/MEOPAR/Karyn/01jan08_Thrml19/SalishSea_1m_{datestr}_{datestr}'
        
        with xr.open_dataset(prefix + '_grid_T.nc') as ds:
            q = ds.votemper.isel(deptht=0, **slc).values
            q2 = q[0,:,:]
            monthly_array_temp_exp_slice[year-2007,month-1,:,:] = q2 #year2007 is index 0 along 1st dimension
            for var in ['votemper']:
                data[var].append(ds.votemper.isel(deptht=0, **slc).values)
    # Concatenate months
    for var in variables: aggregates[var][year] = np.concatenate(data[var]).mean(axis=0)

for year in [2008]:
    # Initialize lists
    for var in variables: data[var] = []
    # Load monthly averages
    for month in range(7, 13):
        datestr = f'{year}{month:02d}'
        prefix = f'/data/sallen/results/MEOPAR/Karyn/01jul08_Thrml19/SalishSea_1m_{datestr}_{datestr}'
        
        with xr.open_dataset(prefix + '_grid_T.nc') as ds:
            q = ds.votemper.isel(deptht=0, **slc).values
            q2 = q[0,:,:]
            monthly_array_temp_exp_slice[year-2007,month-1,:,:] = q2 #year2007 is index 0 along 1st dimension
            for var in ['votemper']:
                data[var].append(ds.votemper.isel(deptht=0, **slc).values)
    # Concatenate months
    for var in variables: aggregates[var][year] = np.concatenate(data[var]).mean(axis=0)

In [9]:
monthly_array_temp_exp_slice[monthly_array_temp_exp_slice == 0 ] = np.nan
monthly_array_temp_exp_slicemean = \
np.nanmean(np.nanmean(monthly_array_temp_exp_slice, axis = 2),axis = 2)
print(np.shape(monthly_array_temp_exp_slicemean))

(14, 12)


/tmp/ipykernel_3793737/2338418733.py:3: RuntimeWarning: Mean of empty slice
  np.nanmean(np.nanmean(monthly_array_temp_exp_slice, axis = 2),axis = 2)


In [10]:
monthly_array_temp_exp_slicemean[1,:] ## Experiment 2 SST 

array([ 6.55805313,  4.6761362 ,  6.77186194, 10.27311911, 13.59493183,
       17.42428301, 18.35690238, 18.73194292, 16.01386188, 11.39800505,
        8.80717597,  6.78115408])

In [11]:
monthly_array_temp_orig_slicemean[1,:] ## Original SST

array([ 6.12548543,  5.80986749,  7.48572357,  9.13112748, 12.73411507,
       16.15264921, 18.19417126, 18.16483525, 15.4019242 , 12.00080142,
        9.30544262,  6.33640542])

### Temperature response factors


In [12]:
TdepDiatOrig,__,TdepNanoOrig=calc_T_Factors(monthly_array_temp_orig_slicemean[1,:],nml)
TdepDiatExp,__,TdepNanoExp=calc_T_Factors(monthly_array_temp_exp_slicemean[1,:],nml)

#### Open Nutrient files

In [13]:
## Original CY 


monthly_array_nitrate_orig_slice = np.zeros([14,12,50,50])
# Load monthly averages
mask = xr.open_dataset('/data/eolson/results/MEOPAR/NEMO-forcing-new/grid/mesh_mask201702.nc')
slc = {'y': slice(450,500), 'x': slice(250,300)} 
e3t, tmask = [mask[var].isel(z=slice(None, 27),**slc).values for var in ('e3t_0', 'tmask')]
years, variables = range(2007, 2021), ['nitrate']
# Temporary list dict
data = {}
# Permanent aggregate dict
aggregates = {var: {} for var in variables}
monthlydat = {var: {} for var in variables}
        
# Add experiment year
for year in [2008]:
    # Initialize lists
    for var in variables: data[var] = []
    # Load monthly averages
    for month in range(1, 13):
        datestr = f'{year}{month:02d}'
        prefix = f'/data/sallen/results/MEOPAR/v201905r/SalishSea_1m_{datestr}_{datestr}'
        
        
        # Load grazing variables
        with xr.open_dataset(prefix + '_ptrc_T.nc') as ds:
            q = ds.nitrate.isel(deptht=0, **slc).values
            q2 = q[0,:,:]
            monthly_array_nitrate_orig_slice[year-2007,month-1,:,:] = q2 #year2007 is index 0 along 1st dimension
            for var in ['nitrate']:
                data[var].append(ds.nitrate.isel(deptht=0, **slc).values)
                
    # Concatenate months
    for var in variables: aggregates[var][year] = np.concatenate(data[var]).mean(axis=0)        


In [14]:
monthly_array_nitrate_orig_slice[monthly_array_nitrate_orig_slice == 0 ] = np.nan
monthly_array_nitrate_orig_slicemean = \
np.nanmean(np.nanmean(monthly_array_nitrate_orig_slice, axis = 2),axis = 2)
print(np.shape(monthly_array_nitrate_orig_slicemean))

(14, 12)


/tmp/ipykernel_3793737/3312634990.py:3: RuntimeWarning: Mean of empty slice
  np.nanmean(np.nanmean(monthly_array_nitrate_orig_slice, axis = 2),axis = 2)


In [15]:
## Experiment 8

monthly_array_nitrate_exp_slice = np.zeros([14,12,50,50])
# Load monthly averages
mask = xr.open_dataset('/data/eolson/results/MEOPAR/NEMO-forcing-new/grid/mesh_mask201702.nc')
slc = {'y': slice(450,500), 'x': slice(250,300)} 
e3t, tmask = [mask[var].isel(z=slice(None, 27),**slc).values for var in ('e3t_0', 'tmask')]
years, variables = range(2007, 2021), ['nitrate']
# Temporary list dict
data = {}
# Permanent aggregate dict
aggregates = {var: {} for var in variables}
monthlydat = {var: {} for var in variables}
        
# Add experiment year
for year in [2008]:
    # Initialize lists
    for var in variables: data[var] = []
    # Load monthly averages
    for month in range(1, 7):
        datestr = f'{year}{month:02d}'
        prefix = f'/data/sallen/results/MEOPAR/Karyn/01jan08_Thrml19/SalishSea_1m_{datestr}_{datestr}'
        
        
        # Load grazing variables
        with xr.open_dataset(prefix + '_ptrc_T.nc') as ds:
            q = ds.nitrate.isel(deptht=0, **slc).values
            q2 = q[0,:,:]
            monthly_array_nitrate_exp_slice[year-2007,month-1,:,:] = q2 #year2007 is index 0 along 1st dimension
            for var in ['nitrate']:
                data[var].append(ds.nitrate.isel(deptht=0, **slc).values)
                
    # Concatenate months
    for var in variables: aggregates[var][year] = np.concatenate(data[var]).mean(axis=0)   
        

# Add experiment year
for year in [2008]:
    # Initialize lists
    for var in variables: data[var] = []
    # Load monthly averages
    for month in range(7, 13):
        datestr = f'{year}{month:02d}'
        prefix = f'/data/sallen/results/MEOPAR/Karyn/01jul08_Thrml19/SalishSea_1m_{datestr}_{datestr}'
        
        
        # Load grazing variables
        with xr.open_dataset(prefix + '_ptrc_T.nc') as ds:
            q = ds.nitrate.isel(deptht=0, **slc).values
            q2 = q[0,:,:]
            monthly_array_nitrate_exp_slice[year-2007,month-1,:,:] = q2 #year2007 is index 0 along 1st dimension
            for var in ['nitrate']:
                data[var].append(ds.nitrate.isel(deptht=0, **slc).values)
                
    # Concatenate months
    for var in variables: aggregates[var][year] = np.concatenate(data[var]).mean(axis=0) 


In [16]:
monthly_array_nitrate_exp_slice[monthly_array_nitrate_exp_slice == 0 ] = np.nan
monthly_array_nitrate_exp_slicemean = \
np.nanmean(np.nanmean(monthly_array_nitrate_exp_slice, axis = 2),axis = 2)
print(np.shape(monthly_array_nitrate_exp_slicemean))

(14, 12)


/tmp/ipykernel_3793737/1010059431.py:3: RuntimeWarning: Mean of empty slice
  np.nanmean(np.nanmean(monthly_array_nitrate_exp_slice, axis = 2),axis = 2)


In [17]:
## Original CY

monthly_array_silicon_orig_slice = np.zeros([14,12,50,50])
# Load monthly averages
mask = xr.open_dataset('/data/eolson/results/MEOPAR/NEMO-forcing-new/grid/mesh_mask201702.nc')
slc = {'y': slice(450,500), 'x': slice(250,300)} 
e3t, tmask = [mask[var].isel(z=slice(None, 27),**slc).values for var in ('e3t_0', 'tmask')]
years, variables = range(2007, 2021), ['silicon']
# Temporary list dict
data = {}
# Permanent aggregate dict
aggregates = {var: {} for var in variables}
monthlydat = {var: {} for var in variables}

        
# Add experiment year
for year in [2008]:
    # Initialize lists
    for var in variables: data[var] = []
    # Load monthly averages
    for month in range(1, 13):
        datestr = f'{year}{month:02d}'
        prefix = f'/data/sallen/results/MEOPAR/v201905r/SalishSea_1m_{datestr}_{datestr}'
        
        
        # Load grazing variables
        with xr.open_dataset(prefix + '_ptrc_T.nc') as ds:
            q = ds.silicon.isel(deptht=0, **slc).values
            q2 = q[0,:,:]
            monthly_array_silicon_orig_slice[year-2007,month-1,:,:] = q2 #year2007 is index 0 along 1st dimension
            for var in ['silicon']:
                data[var].append(ds.silicon.isel(deptht=0, **slc).values)
    
    # Concatenate months
    for var in variables: aggregates[var][year] = np.concatenate(data[var]).mean(axis=0)        

       

In [18]:
monthly_array_silicon_orig_slice[monthly_array_silicon_orig_slice == 0 ] = np.nan
monthly_array_silicon_orig_slicemean = \
np.nanmean(np.nanmean(monthly_array_silicon_orig_slice, axis = 2),axis = 2)
print(np.shape(monthly_array_silicon_orig_slicemean))

(14, 12)


/tmp/ipykernel_3793737/241793216.py:3: RuntimeWarning: Mean of empty slice
  np.nanmean(np.nanmean(monthly_array_silicon_orig_slice, axis = 2),axis = 2)


In [19]:
## Experiment 8

monthly_array_silicon_exp_slice = np.zeros([14,12,50,50])
# Load monthly averages
mask = xr.open_dataset('/data/eolson/results/MEOPAR/NEMO-forcing-new/grid/mesh_mask201702.nc')
slc = {'y': slice(450,500), 'x': slice(250,300)} 
e3t, tmask = [mask[var].isel(z=slice(None, 27),**slc).values for var in ('e3t_0', 'tmask')]
years, variables = range(2007, 2021), ['silicon']
# Temporary list dict
data = {}
# Permanent aggregate dict
aggregates = {var: {} for var in variables}
monthlydat = {var: {} for var in variables}
        
# Add experiment year
for year in [2008]:
    # Initialize lists
    for var in variables: data[var] = []
    # Load monthly averages
    for month in range(1, 7):
        datestr = f'{year}{month:02d}'
        prefix = f'/data/sallen/results/MEOPAR/Karyn/01jan08_Thrml19/SalishSea_1m_{datestr}_{datestr}'
        
        
        # Load grazing variables
        with xr.open_dataset(prefix + '_ptrc_T.nc') as ds:
            q = ds.silicon.isel(deptht=0, **slc).values
            q2 = q[0,:,:]
            monthly_array_silicon_exp_slice[year-2007,month-1,:,:] = q2 #year2007 is index 0 along 1st dimension
            for var in ['silicon']:
                data[var].append(ds.silicon.isel(deptht=0, **slc).values)
                
    # Concatenate months
    for var in variables: aggregates[var][year] = np.concatenate(data[var]).mean(axis=0)   
        

# Add experiment year
for year in [2008]:
    # Initialize lists
    for var in variables: data[var] = []
    # Load monthly averages
    for month in range(7, 13):
        datestr = f'{year}{month:02d}'
        prefix = f'/data/sallen/results/MEOPAR/Karyn/01jul08_Thrml19/SalishSea_1m_{datestr}_{datestr}'
        
        
        # Load grazing variables
        with xr.open_dataset(prefix + '_ptrc_T.nc') as ds:
            q = ds.silicon.isel(deptht=0, **slc).values
            q2 = q[0,:,:]
            monthly_array_silicon_exp_slice[year-2007,month-1,:,:] = q2 #year2007 is index 0 along 1st dimension
            for var in ['silicon']:
                data[var].append(ds.silicon.isel(deptht=0, **slc).values)
                
    # Concatenate months
    for var in variables: aggregates[var][year] = np.concatenate(data[var]).mean(axis=0) 


In [20]:
monthly_array_silicon_exp_slice[monthly_array_silicon_exp_slice == 0 ] = np.nan
monthly_array_silicon_exp_slicemean = \
np.nanmean(np.nanmean(monthly_array_silicon_exp_slice, axis = 2),axis = 2)
print(np.shape(monthly_array_silicon_exp_slicemean))

(14, 12)


/tmp/ipykernel_3793737/4227105976.py:3: RuntimeWarning: Mean of empty slice
  np.nanmean(np.nanmean(monthly_array_silicon_exp_slice, axis = 2),axis = 2)


In [21]:
## Original CY

monthly_array_ammonium_orig_slice = np.zeros([14,12,50,50])
# Load monthly averages
mask = xr.open_dataset('/data/eolson/results/MEOPAR/NEMO-forcing-new/grid/mesh_mask201702.nc')
slc = {'y': slice(450,500), 'x': slice(250,300)} 
e3t, tmask = [mask[var].isel(z=slice(None, 27),**slc).values for var in ('e3t_0', 'tmask')]
years, variables = range(2007, 2021), ['ammonium']
# Temporary list dict
data = {}
# Permanent aggregate dict
aggregates = {var: {} for var in variables}
monthlydat = {var: {} for var in variables}

        
# Add experiment year
for year in [2008]:
    # Initialize lists
    for var in variables: data[var] = []
    # Load monthly averages
    for month in range(1, 13):
        datestr = f'{year}{month:02d}'
        prefix = f'/data/sallen/results/MEOPAR/v201905r/SalishSea_1m_{datestr}_{datestr}'
        
        
        # Load grazing variables
        with xr.open_dataset(prefix + '_ptrc_T.nc') as ds:
            q = ds.ammonium.isel(deptht=0, **slc).values
            q2 = q[0,:,:]
            monthly_array_ammonium_orig_slice[year-2007,month-1,:,:] = q2 #year2007 is index 0 along 1st dimension
            for var in ['ammonium']:
                data[var].append(ds.ammonium.isel(deptht=0, **slc).values)
    
    # Concatenate months
    for var in variables: aggregates[var][year] = np.concatenate(data[var]).mean(axis=0)        

       

In [22]:
monthly_array_ammonium_orig_slice[monthly_array_ammonium_orig_slice == 0 ] = np.nan
monthly_array_ammonium_orig_slicemean = \
np.nanmean(np.nanmean(monthly_array_ammonium_orig_slice, axis = 2),axis = 2)
print(np.shape(monthly_array_ammonium_orig_slicemean))

(14, 12)


/tmp/ipykernel_3793737/3976818894.py:3: RuntimeWarning: Mean of empty slice
  np.nanmean(np.nanmean(monthly_array_ammonium_orig_slice, axis = 2),axis = 2)


In [23]:
## Experiment 8


monthly_array_ammonium_exp_slice = np.zeros([14,12,50,50])
# Load monthly averages
mask = xr.open_dataset('/data/eolson/results/MEOPAR/NEMO-forcing-new/grid/mesh_mask201702.nc')
slc = {'y': slice(450,500), 'x': slice(250,300)} 
e3t, tmask = [mask[var].isel(z=slice(None, 27),**slc).values for var in ('e3t_0', 'tmask')]
years, variables = range(2007, 2021), ['ammonium']
# Temporary list dict
data = {}
# Permanent aggregate dict
aggregates = {var: {} for var in variables}
monthlydat = {var: {} for var in variables}
        
# Add experiment year
for year in [2008]:
    # Initialize lists
    for var in variables: data[var] = []
    # Load monthly averages
    for month in range(1, 7):
        datestr = f'{year}{month:02d}'
        prefix = f'/data/sallen/results/MEOPAR/Karyn/01jan08_Thrml19/SalishSea_1m_{datestr}_{datestr}'
        
        
        # Load grazing variables
        with xr.open_dataset(prefix + '_ptrc_T.nc') as ds:
            q = ds.ammonium.isel(deptht=0, **slc).values
            q2 = q[0,:,:]
            monthly_array_ammonium_exp_slice[year-2007,month-1,:,:] = q2 #year2007 is index 0 along 1st dimension
            for var in ['ammonium']:
                data[var].append(ds.ammonium.isel(deptht=0, **slc).values)
                
    # Concatenate months
    for var in variables: aggregates[var][year] = np.concatenate(data[var]).mean(axis=0)   
        

# Add experiment year
for year in [2008]:
    # Initialize lists
    for var in variables: data[var] = []
    # Load monthly averages
    for month in range(7, 13):
        datestr = f'{year}{month:02d}'
        prefix = f'/data/sallen/results/MEOPAR/Karyn/01jul08_Thrml19/SalishSea_1m_{datestr}_{datestr}'
        
        
        # Load grazing variables
        with xr.open_dataset(prefix + '_ptrc_T.nc') as ds:
            q = ds.ammonium.isel(deptht=0, **slc).values
            q2 = q[0,:,:]
            monthly_array_ammonium_exp_slice[year-2007,month-1,:,:] = q2 #year2007 is index 0 along 1st dimension
            for var in ['ammonium']:
                data[var].append(ds.ammonium.isel(deptht=0, **slc).values)
                
    # Concatenate months
    for var in variables: aggregates[var][year] = np.concatenate(data[var]).mean(axis=0) 


In [24]:
monthly_array_ammonium_exp_slice[monthly_array_ammonium_exp_slice == 0 ] = np.nan
monthly_array_ammonium_exp_slicemean = \
np.nanmean(np.nanmean(monthly_array_ammonium_exp_slice, axis = 2),axis = 2)
print(np.shape(monthly_array_ammonium_exp_slicemean))

(14, 12)


/tmp/ipykernel_3793737/3603250393.py:3: RuntimeWarning: Mean of empty slice
  np.nanmean(np.nanmean(monthly_array_ammonium_exp_slice, axis = 2),axis = 2)


In [25]:
# for now just set light to constant and ignore 'limiter' and 'limval'
DiatLimOrig, __, NanoLimOrig = bt.calc_p_limiters(10*np.ones(np.shape(monthly_array_nitrate_orig_slicemean[1,:])),
                                               NO=monthly_array_nitrate_orig_slicemean[1,:],
                                               NH=monthly_array_ammonium_orig_slicemean[1,:],
                                               Si=monthly_array_silicon_orig_slicemean[1,:],
                                               tmask=np.ones(np.shape(monthly_array_nitrate_orig_slicemean[1,:])),
                                               nampisprod=nml)

In [26]:
# for now just set light to constant and ignore 'limiter' and 'limval'
DiatLimExp, __, NanoLimExp = bt.calc_p_limiters(10*np.ones(np.shape(monthly_array_nitrate_exp_slicemean[1,:])),
                                               NO=monthly_array_nitrate_exp_slicemean[1,:],
                                               NH=monthly_array_ammonium_exp_slicemean[1,:],
                                               Si=monthly_array_silicon_exp_slicemean[1,:],
                                               tmask=np.ones(np.shape(monthly_array_nitrate_exp_slicemean[1,:])),
                                               nampisprod=nml)

In [27]:
DiatLimOrig

{'ILim': array([0.51559692, 0.51559692, 0.51559692, 0.51559692, 0.51559692,
        0.51559692, 0.51559692, 0.51559692, 0.51559692, 0.51559692,
        0.51559692, 0.51559692]),
 'NLim': array([0.92615692, 0.91989456, 0.91559366, 0.78472399, 0.6766017 ,
        0.48910759, 0.5398265 , 0.69177229, 0.82237917, 0.8856945 ,
        0.9163549 , 0.92115137]),
 'SiLim': array([0.95793134, 0.95925508, 0.95802654, 0.90905993, 0.86321779,
        0.89763641, 0.85658661, 0.86556353, 0.91291876, 0.93082761,
        0.95026166, 0.95534427]),
 'limiter': array([0., 0., 0., 0., 0., 2., 0., 0., 0., 0., 0., 0.]),
 'limval': array([0.51559692, 0.51559692, 0.51559692, 0.51559692, 0.51559692,
        2.48910759, 0.51559692, 0.51559692, 0.51559692, 0.51559692,
        0.51559692, 0.51559692])}

In [28]:
NutLimDiatOrig=np.where(DiatLimOrig['SiLim']<DiatLimOrig['NLim'],DiatLimOrig['SiLim'],DiatLimOrig['NLim'])
NutLimNanoOrig=NanoLimOrig['NLim']
NutLimDiatExp=np.where(DiatLimExp['SiLim']<DiatLimExp['NLim'],DiatLimExp['SiLim'],DiatLimExp['NLim'])
NutLimNanoExp=NanoLimExp['NLim']

### Now calculate diatom to flagellate overall growth ratios (multiply by max growth) when temperature and nutrient responses are considered in isolation:

###### * these rates would never occur in the model, even under repleat/optimal conditions, as the maximum of the nutrient and temperature responses are not equal to 1.

In [29]:
mumaxDiat=nml['zz_rate_r_diat']
mumaxNano=nml['zz_rate_r_nano']

## Get values to use in summary plots in Figure9_TempNutDependence.ipynb notebook

In [30]:
mumaxDiat/mumaxNano*NutLimDiatExp/NutLimNanoExp #'CY with WY Nutrients Nutrient dependence only'

array([2.72077093, 2.70944657, 2.68710666, 2.24705017, 2.16685761,
       1.55512552, 1.76019174, 1.94827353, 2.50659433, 2.6735886 ,
       2.70614202, 2.71361351])

In [31]:
mumaxDiat/mumaxNano*TdepDiatExp/TdepNanoExp #'CY with WY Nutrients Temperature dependence only'

array([2.86842105, 2.86842105, 2.86842105, 2.86842105, 2.54163991,
       1.7570548 , 1.61017891, 1.57797733, 2.04603206, 2.86842105,
       2.86842105, 2.86842105])

In [32]:
mumaxDiat/mumaxNano*TdepDiatExp/TdepNanoExp*NutLimDiatExp/NutLimNanoExp #'CY with WY Nutrients both'

array([2.72077093, 2.70944657, 2.68710666, 2.24705017, 1.92000117,
       0.95259403, 0.98807796, 1.07178528, 1.78794265, 2.6735886 ,
       2.70614202, 2.71361351])